# Pure data processing:

- Merge 4 datasets
- Create columns for every keyword
- Take the "unemployment Google searches" keyword, remove the first 2 rows of unemployment, slide the rest of the data to fill that holes  


In [1]:
import pandas as pd

In [2]:
df_political=pd.read_csv("./input/dashboard_spanish_news_political.csv.gz",compression='gzip', header=0, quotechar='"', error_bad_lines=False)
df_political.sort_values(by=["Date"],inplace=True)

df_economical=pd.read_csv("./input/dashboard_spanish_news_economical.csv.gz",compression='gzip', header=0, quotechar='"', error_bad_lines=False)
df_economical.sort_values(by=["Date"],inplace=True)

df_social=pd.read_csv("./input/dashboard_spanish_news_social.csv.gz",compression='gzip', header=0, quotechar='"', error_bad_lines=False)
df_social.sort_values(by=["Date"],inplace=True)

df_google=pd.read_csv("./input/data_pytrends.csv")
df_google.sort_values(by=["date"],inplace=True)

## Dataset structures:

- Requested data from BigQuery (df_political/social/economical): Every different keyword in **political** will be a column with its associated sentiment
- The same for google searches: Every different keyword will be a column with its associated trend_index.
- All df will be resampled and grouped by date to have a weekly display with the corresponding average of **sentiment** or **trend_index**

In [3]:
df_political.head()

,political,Date,Sentiment
15773,juicio,2019-01-01,0.18
52050,seguridad_nacional,2019-01-01,-6.55
52051,seguridad_nacional,2019-01-01,-1.43
52052,seguridad_nacional,2019-01-01,-1.43
53246,inestabilidad_politica,2019-01-01,-0.30


In [4]:
# google searches
df_google.columns

Index(['Unnamed: 0', 'keyword', 'date', 'trend_index'], dtype='object')

In [5]:
df_google.drop(columns='Unnamed: 0', inplace=True)
#df_google.rename(columns={'date':"Date"}, inplace=True)
df_google.head()

,keyword,date,trend_index
0,zoom,2019-01-06,5
1410,bildu,2019-01-06,3
6204,uber eats,2019-01-06,10
4982,medico,2019-01-06,89
2632,productividad,2019-01-06,39


## Google Dataset: 

- Creating a column for each keyword with the trend_index value

In [6]:
# new dataframe
df_google_dates=pd.DataFrame()

# creating the Date column in new dataset
df_google_dates["date"]=list(set(df_google["date"]))
df_google_dates["date"]=pd.to_datetime(df_google_dates["date"])
df_google_dates.sort_values(by=["date"],inplace=True)
df_google_dates.head()

,date
68,2019-01-06
60,2019-01-13
78,2019-01-20
54,2019-01-27
53,2019-02-03


In [7]:
# Creating the new columns. Trend index with the name of the corresponding keyword
keyword_list=list(set(df_google["keyword"]))
keyword_list.sort()
for k in keyword_list:
    df_google_dates[k]=df_google[(df_google['keyword'] == k)]["trend_index"].tolist()
df_google_dates.index=df_google_dates["date"]
df_google_dates.drop(columns="date",inplace=True)

In [8]:
df_google_dates.head()

,amazon,autonomo,ayuda alquiler,badi,banco alimentos,barometro,bildu,bullying,cabify,caritas,...,taxi,teletrabajo,tinder,uber,uber eats,videoconferencia,videollamada,vox,yoga,zoom
date,,,,,,,,,,,,,,,,,,,,,
2019-01-06,59,47,38,38,5,32,3,24,20,47,...,42,2,56,27,10,2,2,35,50,5
2019-01-13,50,44,32,51,12,28,4,27,30,47,...,49,1,51,35,15,3,1,27,44,4
2019-01-20,44,41,20,38,13,31,2,25,100,45,...,100,2,50,100,9,3,2,16,41,3
2019-01-27,46,51,19,45,6,44,4,30,68,40,...,85,2,52,78,11,3,3,11,49,4
2019-02-03,48,47,21,32,7,58,5,24,45,42,...,67,1,50,40,11,3,2,13,48,4


In [9]:
# lets check it out if it's right
print(list(df_google_dates["zoom"][:10]),
      "<==>",
      df_google[df_google["keyword"]=="zoom"]["trend_index"].tolist()[:10],
      ", allright then"
     )

[5, 4, 3, 4, 4, 4, 4, 4, 3, 3] <==> [5, 4, 3, 4, 4, 4, 4, 4, 3, 3] , allright then


## Sliding "unemployment" column.

- Now, I have to remove the first 2 rows of the keyword "desempleo", and supress that space with the rest of the column, so the last 2 rows will be empty 

In [10]:
# I should perform feature ingineering before doing this, to check what's really going on

#desempleo_list=list(df_google_dates["desempleo"])

# delete first 0 positions and add empty ones at the end (not the most elegant)
#desempleo_list.pop(0)
#desempleo_list.pop(1)
#desempleo_list.append(0)
#desempleo_list.append(0)

# add to the dataset
#df_google_dates["desempleo"]=desempleo_list

# ok, it works
#df_google_dates[["Date","desempleo"]]

# Manipulating datasets with Spanish news and sentiment.

We'll need to:

- Create a column for each keyword
- Count occurrences of that keyword
- Measure average sentiment
- Group data by week, starting on monday, to merge with the Google dataset
- Combine occurrences and sentiment into one column representative of both, for each keyword

In [11]:
# let's pplay with the 1st dataset and a random keyword, for instance
df_political[df_political["political"]=="juicio"].head()

,political,Date,Sentiment
15773,juicio,2019-01-01,0.18
15774,juicio,2019-01-01,0.18
15784,juicio,2019-01-01,-6.08
15777,juicio,2019-01-01,-4.06
15778,juicio,2019-01-01,-4.06


- So, I need to measure the average of sentiment of each keyword per day

In [12]:
df_political[df_political["political"]=="juicio"].groupby("Date").mean()

,Sentiment
Date,
2019-01-01,-2.930000
2019-01-02,-3.986667
2019-01-03,-2.350000
2019-01-04,-4.822500
2019-01-05,-4.723333
...,...
2020-10-14,-7.511739
2020-10-15,-6.625128
2020-10-16,-8.926000


- Also, counting occurrences of that keyword

In [13]:
df_political[df_political["political"]=="juicio"].groupby("Date").count()

,political,Sentiment
Date,,
2019-01-01,14,14
2019-01-02,18,18
2019-01-03,40,40
2019-01-04,16,16
2019-01-05,60,60
...,...,...
2020-10-14,23,23
2020-10-15,39,39
2020-10-16,5,5


- Let's use an aggregate to perform both

In [14]:
df2=df_political[df_political["political"]=="juicio"].groupby(["Date"]).agg(['count','mean'])
# erase multiindex
df2.columns=df2.columns.droplevel(0)
df2.head()

,count,mean
Date,,
2019-01-01,14,-2.930000
2019-01-02,18,-3.986667
2019-01-03,40,-2.350000
2019-01-04,16,-4.822500
2019-01-05,60,-4.723333


- Great, now let's resample by week, starting on Sunday, like the Google Searches dataset

In [15]:
df2.index = pd.to_datetime(df2.index)
df2 = df2.resample('W-SUN').mean() #weekly totals
# score is how we are going to measure the keywords
df2["score"]=df2["count"]*df2["mean"]
df2.head()

,count,mean,score
Date,,,
2019-01-06,27.666667,-4.097824,-113.373133
2019-01-13,68.857143,-6.805291,-468.592876
2019-01-20,36.857143,-6.178493,-227.721615
2019-01-27,37.714286,-5.767773,-217.527430
2019-02-03,36.000000,-4.441655,-159.899564


ok, now I know how to do it, then let's continue creating a function to perform this for every keyword in every Spanish news dataset

- I need to create an empty dataframe, 
- loop por ,each keyword from a set of keywords
- perform what i did before for all keywords
- concat to the mepty dataframe
- put all this in a function

In [16]:
# pending of erasing this and writing it in a separate script
def creating_dataset(df,column):
    '''
    Column is the column in which are allocated the keywords, for every case: political, social and economical columns
    '''
    
    # list of new columns
    list_keywords=list(set(df[column]))
    # creating empty dataframe to append info
    df_final=pd.DataFrame()
    df_final["date"]=list(set(df["Date"]))
    
    #date1 = '2019-01-06'
    #date2 = max(df["date"])
    #mydates = pd.date_range(date1, date2, freq="D").tolist()
    #df_final=pd.DataFrame()
    #df_final["date"]=mydates
    
    for k in list_keywords:
        # creating a new dataframe for every keyword in the column, getting the occurrences of keyword and mean of sentiment
        df4=pd.DataFrame()
        df4=df[df[column]==k].groupby(["Date"]).agg(['count','mean'])
        # erase multiindex
        df4.columns=df4.columns.droplevel(0)
        # this will be our score, occurrences * mean 
        df4[k]=df4["count"]*df4["mean"]
        # date column to perform the join by it
        df4["date"]=df4.index
        df4.drop(columns=["count","mean"],inplace=True)
        # this is where we combine the empty dataset, every keyword in its place
        df_final=df_final.merge(df4,how='left', left_on='date', right_on='date')

    # make datetime index for weekly resampling
    df_final["date"]=pd.to_datetime(df_final['date']) 
    df_final.index=df_final["date"]
    # resampling
    df_final = df_final.resample('W-SUN').mean() #weekly totals
    df_final.sort_values(by="date", ascending=True, inplace=True)
    # filling gaps
    df_final=df_final.fillna(0)
    
    # this is for the future join
    
    return df_final

In [17]:
#from my_functions import creating_dataset

In [18]:
dfp = creating_dataset(df_political,"political")
dfs = creating_dataset(df_social,"social")
dfe = creating_dataset(df_economical,"economical")

In [19]:
dfp.head()

,corrupcion,rebelion,vigilancia,seguridad_nacional,ejercito,terrorismo,precio_petroleo,inestabilidad_politica,juicio,refugiados,protestas,extremismo
date,,,,,,,,,,,,
2019-01-06,-92.164000,-21.453333,-31.525,-30.555000,-20.473333,-82.140000,0.00,-7.770000,-111.880000,-6.425000,-0.986667,-29.993333
2019-01-13,-168.828571,-17.413333,-67.150,-5.313333,-31.360000,-11.586667,0.00,-23.500000,-485.394286,-15.266667,-16.754286,-2.260000
2019-01-20,-106.468571,-26.305000,-49.556,-3.220000,-23.700000,-145.260000,3.72,-36.120000,-232.251429,0.000000,-98.830000,-5.644000
2019-01-27,-106.342857,-26.570000,-14.530,-16.080000,-25.508571,-34.084000,0.00,-24.146667,-233.122857,1.276000,-41.448571,-4.226667
2019-02-03,-112.497143,-28.106667,-21.495,-1.480000,-6.876667,-24.512000,0.00,-5.160000,-173.451429,-20.310000,-35.362857,0.000000


In [20]:
dfs.head()

,enfermedades_muy_infecciosas,emprendimiento,vacunas,racismo,inmigracion,ciencia,precio_vivienda,censura_en_medios,subsidios,energias_renovables,agresion_sexual,emergencia_sanitaria
date,,,,,,,,,,,,
2019-01-06,-195.310000,1.680,-23.89,0.0,-1.450000,-6.680000,0.00,0.00,0.0,0.00,-52.646667,-88.380000
2019-01-13,-69.154286,5.300,-11.04,0.0,0.896667,-36.077143,-2.64,0.00,0.0,0.00,-109.376000,-44.537143
2019-01-20,-109.811429,0.000,-5.89,0.0,3.912000,-7.240000,0.00,0.00,0.0,0.00,-8.700000,-56.360000
2019-01-27,-101.220000,-3.932,0.00,0.0,-6.730000,-26.273333,0.00,0.00,0.0,0.00,-17.140000,-28.240000
2019-02-03,-120.577143,5.640,0.00,0.0,-79.540000,-8.630000,0.34,1.74,0.0,-2.78,-164.080000,-56.934286


In [21]:
dfe.head()

,libre_comercio,pobreza,banco_mundial,crecimiento_economico,quiebra_economica,finanzas_y_bancos,inflacion_economica,precio_petroleo,desempleo,job_quality_&_labor_market_performance,stock_market,incertidumbre_economica,macroeconomia_deuda_y_vulnerabilidad,prosperidad_economica_y_finanzas
date,,,,,,,,,,,,,,
2019-01-06,0.00,-41.553333,0.0,0.00,0.000000,0.0,0.00,0.00,-41.995000,0.000000,-19.043333,-631.063333,-37.705,0.0
2019-01-13,-17.82,-66.840000,0.0,0.00,0.000000,0.0,0.00,0.00,-5.933333,-9.820000,-14.948571,-750.242857,-7.460,0.0
2019-01-20,0.00,-23.345714,0.0,0.00,-11.610000,0.0,7.44,3.72,-35.166667,-2.368000,-7.531429,-791.151429,-7.760,0.0
2019-01-27,0.00,-12.205714,0.0,1.99,0.780000,0.0,0.00,0.00,-14.246667,-10.792000,-16.442857,-770.017143,-21.000,0.0
2019-02-03,0.00,-41.560000,-0.5,0.00,-8.773333,0.0,0.00,0.00,-11.875000,-3.333333,-12.111429,-696.837143,-29.860,0.0


# Merging all datasets

In [22]:
from datetime import datetime, date

In [23]:
# creating final dataset with everything
date1 = '2019-01-06'
date2 = datetime.now().date()
mydates = pd.date_range(date1, date2, freq="W").tolist()
df_final=pd.DataFrame()
df_final["date"]=mydates
df_final.head()

,date
0,2019-01-06
1,2019-01-13
2,2019-01-20
3,2019-01-27
4,2019-02-03


In [24]:
dfs = [ dfp, dfe, dfs, df_google_dates] 

In [25]:
# i need to concat instead of merging
'''for d in dfs:
    df_final=df_final.merge(dfe,how='left', left_on='date', right_on='date')
df_final=df_final.fillna(0)'''


"for d in dfs:\n    df_final=df_final.merge(dfe,how='left', left_on='date', right_on='date')\ndf_final=df_final.fillna(0)"

In [26]:
df_final.head()

,date
0,2019-01-06
1,2019-01-13
2,2019-01-20
3,2019-01-27
4,2019-02-03


In [27]:
df_final.columns

Index(['date'], dtype='object')